In [1]:
import numpy as np 
import pandas as pd
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation


train_df = pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/train_data.csv')
test_df = pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/test_data.csv')

# drop features that have no use for the task
train_df = train_df.drop(['case_id'], axis=1)
test_df = test_df.drop(['case_id'], axis=1)

# remove null values
train_df = train_df.dropna()
test_df = test_df.dropna()

# folds in repeated patient ids instances  into N number of visits 
train_df['Visits'] = train_df.groupby(['patientid'])['patientid'].transform('count')
train_df = train_df.drop(['patientid'], axis=1) 
test_df['Visits'] = test_df.groupby(['patientid'])['patientid'].transform('count')
test_df = test_df.drop(['patientid'], axis=1) 

#  save target column
independent_variable_df = train_df['Stay'].copy()
train_df = train_df.drop(['Stay'], axis = 1)

# # create a larger dataset to work with
train_df['Dataset_source'] = 'train'
test_df['Dataset_source'] = 'test'
combined_df = pd.concat([train_df, test_df])

In [2]:
combined_df.info()

In [3]:
def normalize_column(dataframe, column, threshold):

    normalized_values = dataframe[column].value_counts(normalize= True)
    values_below_threshold = normalized_values.where(normalized_values < threshold).dropna().index.values
    dataframe[column] = np.where(dataframe[column].isin(values_below_threshold), '0', dataframe[column])
    

normalize_column(combined_df, 'Hospital_code', 0.05)
normalize_column(combined_df, 'City_Code_Patient', 0.05)

In [4]:
# prepare columns to ease the encoding of categorical values
target=['Hospital_code', 'City_Code_Hospital',
        'City_Code_Patient', 'Severity of Illness',
        'Hospital_type_code', 'Hospital_region_code', 
        'Department', 'Type of Admission',
        'Ward_Facility_Code',  'Ward_Type', 'Age'] 
combined_df[target] = combined_df[target].astype('category')

In [5]:
combined_df.info()

# Encoding

In [6]:
# integer encoding
def encode_columns(dataframe, column_list):
    encoder = LabelEncoder()
    df = dataframe.copy()
    
    for column in column_list:
        df[column] = encoder.fit_transform(df[column])
        
    return df

encode_targets = ['Hospital_code','Hospital_type_code','City_Code_Hospital', 
                    'Hospital_region_code','Department','Ward_Type','Ward_Facility_Code', 
                    'City_Code_Patient', 'City_Code_Hospital', 'Type of Admission', 
                    'Severity of Illness', 'Age' ]


encoded_df = encode_columns(combined_df, encode_targets)



In [7]:
def onehotencode_column(dataframe, column):
    df = dataframe.copy()
    dummies = pd.get_dummies(df[column])
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
    return df

# Training

In [8]:
def dataframe_to_tensor_dataset(dataframe, shuffle = True, batch_size = 32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        ds = ds.prefetch(batch_size)
    return ds

In [9]:
training_set, validation_set, test_set = np.split(combined_df.sample(frac=1), [int(0.7*len(combined_df)), int(0.85*len(combined_df))])

In [10]:
print(f'Length of Training set is {len(training_set)}')
print(f'Length of Validation set is {len(validation_set)}')
print(f'Length of Test set is {len(test_set)}')